In [44]:
import pandas as pd
import json

In [45]:
raw_data = pd.read_csv('../../data/movies.csv')
print(raw_data.shape)
print(raw_data.dtypes)
raw_data.head()

(4000, 41)
adult                                       bool
backdrop_path                             object
genre_ids                                 object
id                                         int64
original_language                         object
original_title                            object
overview                                  object
popularity                               float64
poster_path                               object
release_date                              object
title                                     object
video                                       bool
vote_average                             float64
vote_count                                 int64
budget                                     int64
genres                                    object
imdb_id                                   object
production_companies                      object
production_countries                      object
revenue                                    int64
runtime  

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,...,latin_america_gross,china_opening,china_gross,domestic,international,worldwide,by_release_opening,by_release_gross,by_region_opening,by_region_gross
0,False,/3V4kLQg0kSqPLctI5ziYWabAZYF.jpg,"[878, 28, 12]",912649,en,Venom: The Last Dance,Eddie and Venom are on the run. Hunted by both...,3197.278,/aosm8NMQ3UyoBVpSxyimorCQykC.jpg,2024-10-22,...,20365630.0,0.0,14700000.0,128094490.0,308500000.0,436594490.0,NaN,NaN,NaN,NaN
1,False,/18TSJF1WLA4CkymvVUcKDBwUJ9F.jpg,"[27, 53, 9648]",1034541,en,Terrifier 3,Five years after surviving Art the Clown's Hal...,2204.248,/l1175hgL5DoXnqeZQCcU3eZIdhX.jpg,2024-10-09,...,3556242.0,NaN,NaN,53981071.0,31395883.0,85376954.0,NaN,NaN,NaN,NaN
2,False,/v9acaWVVFdZT5yAU7J2QjwfhXyD.jpg,"[16, 878, 10751]",1184918,en,The Wild Robot,"After a shipwreck, an intelligent robot called...",1868.856,/wTnV3PCVW5O92JMrFvvrRcV39RU.jpg,2024-09-12,...,42402473.0,3207589.0,7133654.0,137913995.0,171143000.0,309056995.0,NaN,NaN,NaN,NaN
3,False,/2fxnTXr8NwyTFkunkimJkGkhqfy.jpg,"[18, 28, 27]",1118031,es,Apocalipsis Z: el principio del fin,When a kind of rabies that transforms people i...,1861.349,/wIGJnIFQlESkC2rLpfA8EDHqk4g.jpg,2024-10-04,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,False,/8mjYwWT50GkRrrRdyHzJorfEfcl.jpg,"[28, 12, 18]",558449,en,Gladiator II,Years after witnessing the death of the revere...,1978.412,/2cxhvwyEwRlysAmRH4iodkvo0z5.jpg,2024-11-13,...,8405072.0,NaN,NaN,0.0,87000000.0,87000000.0,NaN,NaN,NaN,NaN


In [46]:
columns_to_drop = [
    "adult",
    "video",
    "overview",
    "original_language",
    "by_release_opening",
    "by_release_gross",
    "by_region_opening",
    "by_region_gross",
    "genres",
    "china_opening",
    "china_gross",
    "poster_path",
    "status",
    "backdrop_path",
    "poster_path",
    "domestic_gross",
    "asia_pacific_gross",
    "europe_middle_east_and_africa_gross",
    "latin_america_opening",
    "latin_america_gross",
    "tagline",
    "spoken_languages",
    "revenue",
]

raw_data = raw_data.drop(columns=columns_to_drop)

print(raw_data.shape)

raw_data.head()

(4000, 19)


,genre_ids,id,original_title,popularity,release_date,title,vote_average,vote_count,budget,imdb_id,production_companies,production_countries,runtime,domestic_opening,asia_pacific_opening,europe_middle_east_and_africa_opening,domestic,international,worldwide
0,"[878, 28, 12]",912649,Venom: The Last Dance,3197.278,2024-10-22,Venom: The Last Dance,6.500,759,120000000,tt16366836,"Columbia Pictures, Pascal Pictures, Matt Tolma...",United States of America,109,51012404.0,12669884.0,27725511.0,128094490.0,308500000.0,436594490.0
1,"[27, 53, 9648]",1034541,Terrifier 3,2204.248,2024-10-09,Terrifier 3,6.925,1012,2000000,tt27911000,"Cineverse, Bloody Disgusting, Dark Age Cinema,...",United States of America,125,18928113.0,1201876.0,10667946.0,53981071.0,31395883.0,85376954.0
2,"[16, 878, 10751]",1184918,The Wild Robot,1868.856,2024-09-12,The Wild Robot,8.500,2899,78000000,tt29623480,DreamWorks Animation,United States of America,102,35790150.0,4513231.0,16983417.0,137913995.0,171143000.0,309056995.0
3,"[18, 28, 27]",1118031,Apocalipsis Z: el principio del fin,1861.349,2024-10-04,Apocalypse Z: The Beginning of the End,6.769,485,0,tt27599851,Nostromo Pictures,Spain,119,NaN,NaN,NaN,0.0,0.0,0.0
4,"[28, 12, 18]",558449,Gladiator II,1978.412,2024-11-13,Gladiator II,6.800,400,310000000,tt9218128,"Paramount Pictures, Red Wagon Entertainment, P...",United States of America,148,NaN,9200000.0,49245191.0,0.0,87000000.0,87000000.0


In [47]:
drop_index = raw_data[(raw_data['worldwide'] == 0) | (raw_data['budget'] == 0)].index
raw_data = raw_data.drop(drop_index, axis=0).reset_index(drop=True)
print(raw_data.shape)
raw_data.dropna(inplace=True)

(2318, 19)


In [48]:
with open('../../data/genres.json') as f:
    genre_ids = {i['id']: 'genre_' + i['name'].replace(' ', '_') for i in json.load(f)['genres']}
for id, genre in genre_ids.items():
    raw_data[genre] = raw_data['genre_ids'].apply(lambda x: str(id) in x)
raw_data

,genre_ids,id,original_title,popularity,release_date,title,vote_average,vote_count,budget,imdb_id,...,genre_History,genre_Horror,genre_Music,genre_Mystery,genre_Romance,genre_Science_Fiction,genre_TV_Movie,genre_Thriller,genre_War,genre_Western
0,"[878, 28, 12]",912649,Venom: The Last Dance,3197.278,2024-10-22,Venom: The Last Dance,6.500,759,120000000,tt16366836,...,False,False,False,False,False,True,False,False,False,False
1,"[27, 53, 9648]",1034541,Terrifier 3,2204.248,2024-10-09,Terrifier 3,6.925,1012,2000000,tt27911000,...,False,True,False,True,False,False,False,True,False,False
2,"[16, 878, 10751]",1184918,The Wild Robot,1868.856,2024-09-12,The Wild Robot,8.500,2899,78000000,tt29623480,...,False,False,False,False,False,True,False,False,False,False
4,"[28, 35, 878]",533535,Deadpool & Wolverine,1490.952,2024-07-24,Deadpool & Wolverine,7.700,5465,200000000,tt6263850,...,False,False,False,False,False,True,False,False,False,False
5,"[18, 27, 878]",933260,The Substance,1356.547,2024-09-07,The Substance,7.273,2048,17500000,tt17526714,...,False,True,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2310,"[14, 27, 9648]",542224,Gretel & Hansel,25.433,2020-01-30,Gretel & Hansel,5.991,1406,5000000,tt9086228,...,False,True,False,True,False,False,False,False,False,False
2311,"[14, 28, 53, 878]",8698,The League of Extraordinary Gentlemen,25.408,2003-07-11,The League of Extraordinary Gentlemen,5.968,4001,78000000,tt0311429,...,False,False,False,False,False,True,False,True,False,False
2312,"[28, 12, 18]",4643,The Guardian,25.386,2006-09-28,The Guardian,7.140,1505,10000000,tt0406816,...,False,False,False,False,False,False,False,False,False,False
2313,"[80, 18, 53]",50348,The Lincoln Lawyer,25.383,2011-03-17,The Lincoln Lawyer,7.205,3221,40000000,tt1189340,...,False,False,False,False,False,False,False,True,False,False


In [49]:
raw_data = raw_data.loc[:, (raw_data != 0).any(axis=0)]
cleaned_data = raw_data.drop(columns=['genre_ids'])

In [50]:
# Z-Score Normalization

vote_count_mean = (cleaned_data['vote_count']).mean()
vote_count_std = (cleaned_data['vote_count']).std()

cleaned_data['vote_count'] = (cleaned_data['vote_count'] - vote_count_mean) / vote_count_std
cleaned_data.rename(columns={'vote_count': 'vote_count_zscore'}, inplace=True)


In [51]:
cleaned_data.set_index('id', inplace=True)

In [52]:
cols = [col for col in cleaned_data.columns if col not in ['domestic', 'international', 'worldwide']]
cleaned_data = cleaned_data[cols + ['domestic', 'international', 'worldwide']]

In [53]:
cleaned_data.to_csv('../../data/cleaned_movies.csv', index=False)